In [1]:
import sys
sys.path.append('../') 
%load_ext autoreload
%autoreload 2
import sklearn
import copy
import numpy as np

import seaborn as sns
sns.set()

import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
# from viz import viz
from bokeh.plotting import figure, show, output_notebook, output_file, save
from functions import merge_data
from sklearn.model_selection import RandomizedSearchCV
import load_data


from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import fit_and_predict

/accounts/grad/xsli/.local/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = load_data.load_county_level(data_dir = '../data/')
df = df.sort_values('#Deaths_3/31/2020', ascending=False)
# outcome_cases = load_data.outcome_cases # most recent day
# outcome_deaths = load_data.outcome_deaths
important_vars = load_data.important_keys(df)
very_important_vars = ['PopulationDensityperSqMile2010',
#                        'MedicareEnrollment,AgedTot2017',
                       'PopulationEstimate2018',
                       '#ICU_beds',
                       'MedianAge2010',
                       'Smokers_Percentage',
                       'DiabetesPercentage',
                       'HeartDiseaseMortality',
                        '#Hospitals'
#                        'PopMale60-642010',
#                         'PopFmle60-642010',
#                          'PopMale65-742010',
#                          'PopFmle65-742010',
#                          'PopMale75-842010',
#                          'PopFmle75-842010',
#                          'PopMale>842010',
#                          'PopFmle>842010'
                      ]

In [5]:
max_cases = [max(v) for v in df['cases']]
df['max_cases'] = max_cases
df =  df[df['max_cases'] > 0]

In [6]:
exponential = {'model_type':'exponential'}
shared_exponential = {'model_type':'shared_exponential'}
demographics = {'model_type':'shared_exponential', 'demographic_vars':very_important_vars}
linear = {'model_type':'linear'}

In [ ]:
df = fit_and_predict.fit_and_predict_ensemble(df, 
                                              target_day=np.array(range(1, 8)),
                                              mode='predict_future',
                                              outcome='deaths',
                                              methods=[exponential, 
                                                       shared_exponential,
                                                       demographics,
                                                       linear
                                                         ],
                                                  output_key=f'predicted_deaths_ensemble_all'
                                                   )
df = fit_and_predict.fit_and_predict_ensemble(df, 
                                              target_day=np.array(range(1, 8)),
                                              mode='predict_future',
                                              outcome='deaths',
                                              methods=[exponential, 
                                                       shared_exponential,
                                                       linear
                                                         ],
                                                  output_key=f'predicted_deaths_ensemble_no_demographics_all'
                                                   )
df = fit_and_predict.fit_and_predict_ensemble(df, 
                                              target_day=np.array(range(1, 8)),
                                              mode='predict_future',
                                              outcome='deaths',
                                              methods=[exponential, 
                                                       shared_exponential,
                                                       demographics
                                                         ],
                                                  output_key=f'predicted_deaths_ensemble_no_linear_all'
                                                   )
df = fit_and_predict.fit_and_predict_ensemble(df, 
                                              target_day=np.array(range(1, 8)),
                                              mode='predict_future',
                                              outcome='deaths',
                                              methods=[ 
                                                       shared_exponential,
                                                       demographics,
                                                       linear
                                                         ],
                                                  output_key=f'predicted_deaths_ensemble_no_exponential_all'
                                                   )

In [7]:
all_methods = [exponential, 
               shared_exponential,
               demographics,
               linear]
df = fit_and_predict.fit_and_predict_ensemble(df, 
                                              target_day=np.array(range(1, 8)),
                                              mode='predict_future',
                                              outcome='deaths',
                                              methods=all_methods,
                                              output_key=f'predicted_deaths_ensemble_all'
                                              )
for (i, model) in enumerate(all_methods):
        
    if 'demographic_vars' in model:
        demographic_vars = model['demographic_vars']
    else:
        demographic_vars = []
    
    method = model['model_type']
    print(method)
    df = fit_and_predict.fit_and_predict(df, 
                                         outcome='deaths', 
                                         method=method, 
                                         mode='predict_future', 
                                         target_day=np.array(range(1, 8)),
                                         output_key=f'predicted_deaths_{method}_all',
                                         demographic_vars=demographic_vars)
    print(all_methods[0:i]+all_methods[(i+1):])
    df = fit_and_predict.fit_and_predict_ensemble(df, 
                                              target_day=np.array(range(1, 8)),
                                              mode='predict_future',
                                              outcome='deaths',
                                              methods=all_methods[0:i]+all_methods[(i+1):],
                                              output_key=f'predicted_deaths_ensemble_no_{method}_all'
                                              )

--- Model Contributions ---
{'model_type': 'exponential'}: 0.33158405232057736
{'model_type': 'linear'}: 0.3306937014610934
{'model_type': 'shared_exponential'}: 0.17601947272302437
{'model_type': 'shared_exponential', 'demographic_vars': ['PopulationDensityperSqMile2010', 'PopulationEstimate2018', '#ICU_beds', 'MedianAge2010', 'Smokers_Percentage', 'DiabetesPercentage', 'HeartDiseaseMortality', '#Hospitals']}: 0.1617027734953045
exponential
[{'model_type': 'shared_exponential'}, {'model_type': 'shared_exponential', 'demographic_vars': ['PopulationDensityperSqMile2010', 'PopulationEstimate2018', '#ICU_beds', 'MedianAge2010', 'Smokers_Percentage', 'DiabetesPercentage', 'HeartDiseaseMortality', '#Hospitals']}, {'model_type': 'linear'}]
--- Model Contributions ---
{'model_type': 'linear'}: 0.4978565953580966
{'model_type': 'shared_exponential'}: 0.26194760915433785
{'model_type': 'shared_exponential', 'demographic_vars': ['PopulationDensityperSqMile2010', 'PopulationEstimate2018', '#ICU_b

In [8]:
method_keys = [c for c in df if 'predicted' in c]

In [9]:
for key in method_keys:
    for d in range(1, 8):
        newkey = key[:-3] + str(d)
        df[newkey] = np.array([p[d-1] for p in df[key].values])

In [11]:
[c for c in df if 'predicted' in c]

['predicted_deaths_ensemble_all',
 'predicted_deaths_exponential_all',
 'predicted_deaths_ensemble_no_exponential_all',
 'predicted_deaths_shared_exponential_all',
 'predicted_deaths_ensemble_no_shared_exponential_all',
 'predicted_deaths_linear_all',
 'predicted_deaths_ensemble_no_linear_all',
 'predicted_deaths_ensemble_1',
 'predicted_deaths_ensemble_2',
 'predicted_deaths_ensemble_3',
 'predicted_deaths_ensemble_4',
 'predicted_deaths_ensemble_5',
 'predicted_deaths_ensemble_6',
 'predicted_deaths_ensemble_7',
 'predicted_deaths_exponential_1',
 'predicted_deaths_exponential_2',
 'predicted_deaths_exponential_3',
 'predicted_deaths_exponential_4',
 'predicted_deaths_exponential_5',
 'predicted_deaths_exponential_6',
 'predicted_deaths_exponential_7',
 'predicted_deaths_ensemble_no_exponential_1',
 'predicted_deaths_ensemble_no_exponential_2',
 'predicted_deaths_ensemble_no_exponential_3',
 'predicted_deaths_ensemble_no_exponential_4',
 'predicted_deaths_ensemble_no_exponential_5',


In [14]:
df[[c for c in df if 'predicted' in c]].iloc[0]

predicted_deaths_ensemble_all                          [450.4580874531463, 538.8713889539164, 642.662...
predicted_deaths_exponential_all                       [470.69580158127013, 582.4838290310679, 720.82...
predicted_deaths_ensemble_no_exponential_all           [443.9198391656373, 524.7814102219679, 617.411...
predicted_deaths_shared_exponential_all                [459.77057973689125, 557.8093610088187, 671.72...
predicted_deaths_ensemble_no_shared_exponential_all    [447.2881227239592, 532.4249188117444, 632.770...
predicted_deaths_linear_all                            [420.4999999999999, 473.6999999999999, 526.899...
predicted_deaths_ensemble_no_linear_all                [460.48979039883477, 560.694544906916, 681.426...
predicted_deaths_ensemble_1                                                                      450.458
predicted_deaths_ensemble_2                                                                      538.871
predicted_deaths_ensemble_3                            

In [17]:
preds_df.to_csv("../predictions/prediction_04_01.csv")

In [16]:
method_keys = [c for c in df if 'predicted' in c]
geo = ['countyFIPS', 'CountyNamew/StateAbbrev']
preds_df = df[method_keys + geo]